In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
import imageio

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

In [6]:
#!unzip -uq "/content/drive/My Drive/LeWagon_ODR/preprocessed_images2.zip" -d "/content/drive/My Drive/LeWagon_ODR/"

In [7]:
df = pd.read_csv('/content/drive/My Drive/LeWagon_ODR/full_df_cleaned_v3.csv')

In [8]:
df.head()

,Diagnostic,file,target_init,Patient Age,Patient Sex,Target,tarstr,N,D,G,C,A,H,M,O,filename
0,normal fundus,0_right.jpg,"[1, 0, 0, 0, 0, 0, 0, 0]",69,Female,"[1, 0, 0, 0, 0, 0, 0, 0]",N,1,0,0,0,0,0,0,0,0_right_69_Female_N.jpg
1,normal fundus,1_right.jpg,"[1, 0, 0, 0, 0, 0, 0, 0]",57,Male,"[1, 0, 0, 0, 0, 0, 0, 0]",N,1,0,0,0,0,0,0,0,1_right_57_Male_N.jpg
2,moderate non proliferative retinopathy,2_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",42,Male,"[0, 1, 0, 0, 0, 0, 0, 0]",D,0,1,0,0,0,0,0,0,2_right_42_Male_D.jpg
3,mild nonproliferative retinopathy,4_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",53,Male,"[0, 1, 0, 0, 0, 0, 0, 0]",D,0,1,0,0,0,0,0,0,4_right_53_Male_D.jpg
4,moderate non proliferative retinopathy,5_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",50,Female,"[0, 1, 0, 0, 0, 0, 0, 0]",D,0,1,0,0,0,0,0,0,5_right_50_Female_D.jpg


In [9]:
df_n = df[df['tarstr']=='N']

In [10]:
df_n.shape

(2786, 16)

In [11]:
df_c = df[df['tarstr']=='C']

In [12]:
df_c.shape

(260, 16)

In [13]:
df_N_C = pd.concat([df_n, df_c], ignore_index=True)

In [14]:
df_N_C.shape

(3046, 16)

In [15]:
IMAGE_PATH = '/content/drive/My Drive/LeWagon_ODR/preprocessed_images2/'

In [16]:
df_N_C['filepath'] = IMAGE_PATH + df_N_C['filename']

In [17]:
img_data = []
number_id_nofile = []

for i in range(len(df_N_C)):
  try:
    img_data.append(imageio.imread(df_N_C['filepath'][i]))
  except:
    number_id_nofile.append(df_N_C.index[i])

In [18]:
df_N_C = df_N_C.drop(number_id_nofile)

In [19]:
df_N_C['tarstr'].value_counts()

N    2786
C     260
Name: tarstr, dtype: int64

In [20]:
img_data_array = np.array(img_data)

In [21]:
img_data_array.shape

(3046, 256, 256, 3)

In [22]:
X = img_data_array

In [23]:
y = df_N_C['C']

In [24]:
y.shape

(3046,)

In [25]:
y.value_counts(1)

0    0.914642
1    0.085358
Name: C, dtype: float64

Train Test Split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

In [27]:
X_train = X_train / 255
X_test = X_test / 255

In [28]:
df_N_C['C'].value_counts()

0    2786
1     260
Name: C, dtype: int64

In [29]:
y_train.value_counts(1)

0    0.914634
1    0.085366
Name: C, dtype: float64

In [30]:
# def initialize_model():
#   model = models.Sequential()

#   model.add(layers.Conv2D(32, (3,3), input_shape=(256, 256, 3), activation='relu', padding='same'))
#   model.add(layers.MaxPool2D(pool_size=(2,2)))
#   model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
#   model.add(layers.MaxPool2D(pool_size=(2,2)))
#   model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
#   model.add(layers.MaxPool2D(pool_size=(3,3)))
#   model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
#   model.add(layers.MaxPool2D(pool_size=(3,3)))
#   ### Flattening
#   model.add(layers.Flatten())
#   ### One fully connected
#   model.add(layers.Dense(120, activation='relu'))
#   model.add(layers.Dropout(rate=0.5))
#   model.add(layers.Dense(60, activation='relu'))
#   model.add(layers.Dropout(rate=0.5))
#   model.add(layers.Dense(1, activation='sigmoid'))

#   adam_opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)

#   model.compile(loss='binary_crossentropy', 
#         optimizer=adam_opt,
#         metrics=['accuracy'])

#   return model

In [31]:
# model = initialize_model()
# es = EarlyStopping(patience=20, restore_best_weights=True)

# history = model.fit(X_train, y_train,
#                     validation_split=0.3,
#                     epochs=100,
#                     batch_size=16, 
#                     verbose=1,
#                     callbacks=[es])

In [32]:
# print(model.evaluate(X_test, y_test, verbose=0))

In [33]:
# def plot_history(history, title='', axs=None, exp_name=""):
#     if axs is not None:
#         ax1, ax2 = axs
#     else:
#         f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
#     if len(exp_name) > 0 and exp_name[0] != '_':
#         exp_name = '_' + exp_name
#     ax1.plot(history.history['loss'], label='train' + exp_name)
#     ax1.plot(history.history['val_loss'], label='val' + exp_name)
#     ax1.set_ylim(0., 2.2)
#     ax1.set_title('loss')
#     ax1.legend()

#     ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
#     ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
#     ax2.set_ylim(0.25, 1.)
#     ax2.set_title('Accuracy')
#     ax2.legend()
#     return (ax1, ax2)

# # YOUR CODE HERE
# plot_history(history, title='', axs=None, exp_name="")

In [34]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():
    
  # YOUR CODE HERE
  model = VGG16(include_top=False, weights='imagenet',
                input_shape=(256, 256, 3)
                  )
    
  return model

model = load_model()

In [35]:
def set_nontrainable_layers(model):
    # YOUR CODE HERE
  model.trainable=False
  return model

In [36]:
def add_last_layers(model):
  # YOUR CODE HERE
  base_model = set_nontrainable_layers(model)
  flattening_layer = layers.Flatten()
  dense_layer = layers.Dense(500, activation='relu')
  prediction_layer = layers.Dense(1, activation='sigmoid')
    
  model = models.Sequential([
    base_model,
    flattening_layer,
    dense_layer,
    prediction_layer
  ])

  return model

In [37]:
model = load_model()
model = add_last_layers(model)

In [38]:
def compile_model(model):
  adam_opt = optimizers.Adam(learning_rate=0.0001)

  model.compile(loss='binary_crossentropy',
              optimizer=adam_opt,
              metrics=['accuracy'])
  return model

model = compile_model(model)

In [39]:
def build_model():
    # YOUR CODE HERE
    model = load_model()
    model = add_last_layers(model)
    model = compile_model(model)

    return model

model = build_model()

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               16384500  
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total params: 31,099,689
Trainable params: 16,385,001
Non-trainable params: 14,714,688
_________________________________________________________________


In [41]:
es = EarlyStopping(patience=20,  restore_best_weights=True)

model = build_model()

history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    epochs=100,
                    batch_size=16, 
                    verbose=1, 
                    callbacks=[es])

Epoch 1/100
94/94 [==============================] - 27s 141ms/step - loss: 0.4343 - accuracy: 0.8804 - val_loss: 0.1246 - val_accuracy: 0.9281
Epoch 2/100
94/94 [==============================] - 11s 115ms/step - loss: 0.1337 - accuracy: 0.9422 - val_loss: 0.0858 - val_accuracy: 0.9719
Epoch 3/100
94/94 [==============================] - 11s 115ms/step - loss: 0.1090 - accuracy: 0.9482 - val_loss: 0.0918 - val_accuracy: 0.9609
Epoch 4/100
94/94 [==============================] - 11s 116ms/step - loss: 0.1074 - accuracy: 0.9512 - val_loss: 0.0856 - val_accuracy: 0.9609
Epoch 5/100
94/94 [==============================] - 11s 117ms/step - loss: 0.0900 - accuracy: 0.9599 - val_loss: 0.0917 - val_accuracy: 0.9578
Epoch 6/100
94/94 [==============================] - 11s 118ms/step - loss: 0.0899 - accuracy: 0.9602 - val_loss: 0.0783 - val_accuracy: 0.9656
Epoch 7/100
94/94 [==============================] - 11s 118ms/step - loss: 0.0855 - accuracy: 0.9685 - val_loss: 0.0896 - val_accuracy:

In [47]:
import joblib

In [49]:
filename = '/content/drive/My Drive/LeWagon_ODR/test_model_N_C_VGG16.joblib'
joblib.dump(model, filename)

TypeError: ignored

In [50]:
model.evaluate(X_test, y_test)

29/29 [==============================] - 16s 296ms/step - loss: 0.0841 - accuracy: 0.9726


[0.08414428681135178, 0.9726477265357971]